First install and import some necessary Python modules.

In [ ]:
!pip install astropy
!pip install photutils

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.io import ascii
from astropy.stats import sigma_clipped_stats
from photutils import DAOStarFinder
from photutils import CircularAperture
from photutils import aperture_photometry

Open your standard and target FITS files. Make sure you've uploaded them first.

In [ ]:
standard_hdu = fits.open('./standardG_stacked.fit')
target_hdu = fits.open('./SNG_stacked.fit')

Now, we'll get some statistical information about our images. The mean, median and standard deviations of the pixel values in each of your images will print.

In [ ]:
st_data = standard_hdu[0].data
st_mean,st_median,st_std = sigma_clipped_stats(st_data,sigma=3.0)
print(st_mean,st_median,st_std)
SN_data = target_hdu[0].data
SN_mean,SN_median,SN_std = sigma_clipped_stats(SN_data,sigma=3.0)
print(SN_mean,SN_median,SN_std)

Finally, identify the stars in your images. The output from the cell below will be a list of stars. Repeat this process for the image with your standard star and the image with the supernova. Adjust the threshold until your list has a small enough number of stars for you to find the one you're interested in. You may need to increase it quite a bit. Your standard star is located near (3390,1060) in its image and your supernova is located near (2240, 2360) in its image.

In [ ]:
daofind = DAOStarFinder(fwhm=20.0,threshold=5.*st_std)
st_sources = daofind(st_data-st_median)
for col in st_sources.colnames:
    st_sources[col].info.format='%0.8g'
print(st_sources)

In [ ]:
#Repeat for supernova image

Finally, perform aperture photometry to measure the flux for your standard star and your supernova. Change the coordinates of the center of the aperture to be the centroid you identified with DAOfind.

In [ ]:
apertures = CircularAperture([(3390,1060)], r=40.)
st_phot_table = aperture_photometry(st_data-st_median, apertures)
print(st_phot_table)

In [ ]:
#Repeat for supernova image

The flux within your aperture is in the "aperture_sum" column. The exposure time for the standard star was 60s and the exposure time for the supernova was 120s, so double the aperture_sum for the standard star before comparing. Your standard star is Feige 98. Look it up in this table: https://www2.keck.hawaii.edu/inst/common/flux_stds.html to find the apparent V band magnitude. Use this information to calculate the magnitude of the supernova, which is SN 2022hrs, a type Ia supernova in galaxy NGC 4647. Our observation was taken on the night of April 22/23, 2022. Our measurment will be a bit off because of the difference between the V filter and our G filter, but it should be close. You can compare to other measurements here: https://www.rochesterastronomy.org/sn2022/sn2022hrs.html